In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [4]:
path = Path('data/midi')
csv_path = path/'metadata'

In [5]:
source_dir = 'midi_sources'
# source_dir = 'midi_transform_v1'
out_dir = 'midi_transcribe_v1'
source_csv = csv_path/f'{source_dir}.csv'
out_csv = csv_path/f'{out_dir}.csv'

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(source_csv, index_col=0); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,inferred_offset,instruments,midi,midi_title,parts,seconds,section,song_url,source,time_signature,title
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,0.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,yu-gi-oh3,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song
1,weezer,NaN,NaN,128.0,C,NaN,0.0,4.0,NaN,NaN,NaN,data/midi/midi_sources/hooktheory/pianoroll/w/...,My New Song,intro-and-verse,NaN,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,NaN,beverly-hills
2,weezer,108.0,NaN,108.0,Eb,1.0,-3.0,4.0,E- major,-3.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,falling for you intro,"intro,verse,chorus,solo",11.111111,intro,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,falling-for-you
3,weezer,121.0,"Pop,Rock",121.0,Ab,1.0,4.0,4.0,A- major,4.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,Buddy Holly,"verse,pre-chorus,chorus,bridge,solo",43.388430,solo,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,buddy-holly
4,wavves,180.0,NaN,180.0,A,1.0,3.0,4.0,B minor,-2.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,dreams of grandeur,verse,21.333333,verse,https://www.hooktheory.com/theorytab/view/wavv...,hooktheory,4/4,dreams-of-grandeur


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(30201, 20)

In [9]:
df_filtered.head()

,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,inferred_offset,instruments,midi,midi_title,parts,seconds,section,song_url,source,time_signature,title
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,0.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,yu-gi-oh3,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song
2,weezer,108.0,NaN,108.0,Eb,1.0,-3.0,4.0,E- major,-3.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,falling for you intro,"intro,verse,chorus,solo",11.111111,intro,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,falling-for-you
3,weezer,121.0,"Pop,Rock",121.0,Ab,1.0,4.0,4.0,A- major,4.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,Buddy Holly,"verse,pre-chorus,chorus,bridge,solo",43.388430,solo,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,buddy-holly
4,wavves,180.0,NaN,180.0,A,1.0,3.0,4.0,B minor,-2.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,dreams of grandeur,verse,21.333333,verse,https://www.hooktheory.com/theorytab/view/wavv...,hooktheory,4/4,dreams-of-grandeur
6,walter-murphy,126.0,Soundtrack,126.0,C,1.0,0.0,4.0,C major,0.0,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,Freakin FCC Bridge,"intro-and-verse,bridge",17.142857,bridge,https://www.hooktheory.com/theorytab/view/walt...,hooktheory,4/4,the-freaking-fcc


In [11]:
df_filtered.midi.duplicated().any()

False

In [26]:
df_filtered[source_dir].duplicated()

KeyError: 'midi_sources'

In [9]:
from data_sources import process_parallel

In [10]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row[source_dir]
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    try:
        seq = midi2seq(midi_file)
        seq_comp = remove_seq_rests(trim_seq_rests(seq))
        delta_trim = len(seq) - len(seq_comp)
        if delta_trim > 100: print(f'Removed {delta_trim} rests from {midi_file}')
        string_repr = seq2str(seq)
        with open(out_file, 'w') as tf:
            tf.write(string_repr)
    except Exception as e:
        print('Error converting midi to sequence', e)
        return idx,None
    return idx,str(out_file)

In [11]:
# for r in df_filtered.iterrows():
#     transcribe_file(r)

In [ ]:
process_parallel(transcribe_file, df_filtered.iterrows(), total=df_filtered.shape[0])

Error converting midi to sequence index 4707 is out of bounds for axis 0 with size 4707
Error converting midi to sequence index 4707 is out of bounds for axis 0 with size 4707
Error converting midi to sequence index 4707 is out of bounds for axis 0 with size 4707
Removed 229 rests from data/midi/midi_transform_v1/ecomp/2004/ADIG05.mid
Removed 229 rests from data/midi/midi_transform_v1/ecomp/2004/ADIG05.mid
Removed 229 rests from data/midi/midi_transform_v1/ecomp/2004/ADIG05.mid
Removed 229 rests from data/midi/midi_transform_v1/ecomp/2004/ADIG05.mid
Removed 5898 rests from data/midi/midi_transform_v1/ecomp/2009/Toscano01.MID
Removed 5898 rests from data/midi/midi_transform_v1/ecomp/2009/Toscano01.MID
Removed 5898 rests from data/midi/midi_transform_v1/ecomp/2009/Toscano01.MID
Removed 5898 rests from data/midi/midi_transform_v1/ecomp/2009/Toscano01.MID
Removed 6539 rests from data/midi/midi_transform_v1/ecomp/2004/KARYAG03.mid
Removed 6539 rests from data/midi/midi_transform_v1/ecomp/20

In [ ]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=index(transcribed_files.keys()))

In [ ]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

In [ ]:
merged_df.to_csv(out_csv); merged_df.head()